<a href="https://colab.research.google.com/github/sarasabino/Accident_Rate_Project/blob/main/Src/Notebooks/00_Data_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Data Generation

We initially received the data from the accidents occurred in the company, although it is usefull we will need the data from the staff and the resolution if each employee has had an accident or not. This data will allow us to perform our accident prediction model. 

For that purpose we are going to generate a csv file with random data, but based in the real one, that contains the company's staff by employee. Each row represents an employee, with all its characteristic. In the "Accident" column we will have if that person has had an accident during his time in the company or not. 


In [52]:
! cp /content/drive/MyDrive/'Colab Notebooks'/'TFM Accidentabilidad'/Data/HS_Accidentabilidad.csv .
! cp /content/drive/MyDrive/'Colab Notebooks'/'TFM Accidentabilidad'/Data/'G_Plantas y Tech'.csv .
! cp /content/drive/MyDrive/'Colab Notebooks'/'TFM Accidentabilidad'/Data/Datos_plantilla_fabrica.csv .


We are going to upload the accidents' data and filter by year. We would only work with 2018,2019 and 2020 as the other years are incomplete.

In [53]:
import pandas as pd

accidentes = pd.read_csv('HS_Accidentabilidad.csv', sep=';')

accidentes = accidentes[(accidentes['Año']== 2018) | (accidentes['Año']==2019) | (accidentes['Año']==2020) ]

In [54]:
print("We have a total accidents amount of ",  len(accidentes))

We have a total accidents amount of  4180


In [55]:
accidentes.head() ## we have 104 columns which is a lot of information

,ID,- DATOS DEL SUCESO -,Codigo accidentado,Fecha introduccion datos,Fecha accidente,Año,Codigo fecha accidente,Planta,Código empleado,Supervisor,Puesto habitual de trabajo,Puesto habitual de trabajo - Cerrado,MOI o MOD,La estaba desempeñando,"En caso negativo, indicar qué estaba desempeñando",Edad,Género,Turno,Numero personas implicadas,Llevaba EPIs puestos,Hora,Hora jornada,Tipo trabajador,"En caso de ser ajeno, indicar empresa",Lugar o seccion accidente,Línea de producción,Tarea que estaba realizando,Cumplimiento Instruccion de Seguridad,¿Se ha incumplido alguna de las normas o instrucciones existentes y explicadas?,"En caso de incumplimiento, comentarios:","En caso negativo, indicar la razon",Experiencia en puesto,Fecha ultima formacion completa sobre instrucciones en el puesto,Condicion peligrosa,Forma de producirse,Factores de peligro,Acto inseguro,Naturaleza de lesion,- FORMA EN QUE SE PRODUJO -,Descripción supervisor,...,Nombre del Coordinador - despegable,Nombre de participante a la investigación,Responsable de Departamento,Fecha cierre informe,- VALORACIÓN DE LOS HECHOS -,Probabilidad de repeticion del suceso,Gravedad potencial,Podría haberse dado otro riesgo en estas circunstancias,- MEDIDAS DE CONTROL -,Posibilidad de este suceso detectada en la evaluación de riesgos,Estaban contempladas las medidas preventivas en la Instruccion de Seguridad,Los implicados conocian la existencia del riesgo y las medidas a adoptar,"Se habian cumplido las normas establecidas, en caso de estar definidas",- PROPUESTA DE ACCIONES -,Estas acciones pueden añadir riesgos a los ya existentes,"1 En caso afirmativo, cual es",Se abre NC como consecuencia de esta investigacion,"2 En caso afirmativo, cual es",Requiere revision de Evaluacion de Riesgos,"3 En caso afirmativo, cual es",Requiere revision de alguna Instruccion de Seguridad,"4 En caso afirmativo, cual es",Requiere incorporar algun aspecto en el check list de Inspecciones,"5 En caso afirmativo, cual es",Requiere amonestacion verbal el empleado,Bloqueado,Fecha accidente trigger,- ACTUACIONES INMEDIATAS TOMADAS -,Actuación inmediata asistencial,Actuación inmediata instalaciones,- PROPUESTA INICIAL DE MEDIDAS A PLANIFICAR -,Medida a planificar 1,Medida a planificar 2,Medida a planificar 3,Notificacion o investigacion,Modifica notificacion,Se ha dado un suceso similar (en cuanto a sus causas y consecuencias) en el último año en planta?,"En caso afirmativo, cual es y como se actuó?",Tipo Tema,Acta reunión alta baja
2057,6253,0.0,HS-CHGQ_6253,09/03/2019,30/05/2018,2018.0,201805,45,NaN,522,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Turno de mañana,17,NaN,NaN,Primera hora,Incidente (no aplica),NaN,Almacén producto semielaborado o en producción,NaN,NaN,Pendiente incluir el riesgo y medidas en Instr...,NaN,NaN,NaN,NaN,NaN,Deficiente visibilidad del punto de peligro,Choque contra objeto inmóvil,Camiones,Accionar el equipo sin supervisar la maniobra,NaN,0.0,NaN,...,NaN,NaN,Logistics,NaN,0.0,NaN,NaN,NaN,0.0,Si,Si,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,30/05/2018,0.0,NaN,NaN,0.0,NaN,NaN,NaN,No,NaN,NaN,NaN,3,NaN
2062,6813,0.0,HS-CHGQ_6813,09/03/2019,30/05/2018,2018.0,201805,45,NaN,522,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Turno de mañana,17,NaN,NaN,Primera hora,Incidente (no aplica),NaN,Área de embalado,NaN,NaN,Si,NaN,NaN,NaN,NaN,NaN,Ninguna,Choque contra objeto inmóvil,Camiones,Falta de coordinación con el compañero,NaN,0.0,NaN,...,NaN,NaN,Logistics,NaN,0.0,NaN,NaN,NaN,0.0,Si,Si,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,30/05/2018,0.0,NaN,NaN,0.0,NaN,NaN,NaN,No,NaN,NaN,NaN,3,NaN
2063,6814,0.0,HS-CHGQ_6814,09/03/2019,16/07/2018,2018.0,201807,45,NaN,522,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Turno de mañana,17,NaN,NaN,Octava hora,Incidente (no aplica),NaN,Instalaciones exteriores y auxiliares,NaN,NaN,Si,NaN,NaN,NaN,NaN,NaN,"Herramientas o útiles en mal estado, o mal con...",Contacto eléctrico,Otro,Ninguno,NaN,0.0,NaN,...,NaN,NaN,Maintenance,NaN,0.0,NaN,NaN,NaN,0.0,Si,Si,NaN,NaN,0.0,NaN,NaN,NaN,NaN,Na

In [56]:
accidentes.columns

Index(['ID', '-  DATOS DEL SUCESO -', 'Codigo accidentado',
       'Fecha introduccion datos', 'Fecha accidente', 'Año',
       'Codigo fecha accidente', 'Planta', 'Código empleado', 'Supervisor',
       ...
       '- PROPUESTA INICIAL DE MEDIDAS A PLANIFICAR -',
       'Medida a planificar 1', 'Medida a planificar 2',
       'Medida a planificar 3', 'Notificacion o investigacion',
       'Modifica notificacion',
       'Se ha dado un suceso similar (en cuanto a sus causas y consecuencias) en el último año en planta?',
       'En caso afirmativo, cual es y como se actuó?', 'Tipo Tema',
       'Acta reunión alta baja'],
      dtype='object', length=104)

In [57]:
#lets check all columns dtypes
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(accidentes.dtypes)

ID                                                                                                     int64
-  DATOS DEL SUCESO -                                                                                float64
Codigo accidentado                                                                                    object
Fecha introduccion datos                                                                              object
Fecha accidente                                                                                       object
Año                                                                                                  float64
Codigo fecha accidente                                                                                 int64
Planta                                                                                                 int64
Código empleado                                                                                       object
Supervisor         

In [58]:
accidentes['Gravedad potencial'].unique()

array([nan, 'Muy grave', 'Leve', 'Grave'], dtype=object)

In [59]:
## we are going to filter to get the most important columns
staff = accidentes[['Codigo accidentado', 'Puesto habitual de trabajo','Turno','Edad','Género','Planta', 'Tipo accidente', 'Tipo trabajador', 'Gravedad potencial', 'Lugar o seccion accidente']]
staff.head()

,Codigo accidentado,Puesto habitual de trabajo,Turno,Edad,Género,Planta,Tipo accidente,Tipo trabajador,Gravedad potencial,Lugar o seccion accidente
2057,HS-CHGQ_6253,NaN,Turno de mañana,NaN,NaN,45,Incidente,Incidente (no aplica),NaN,Almacén producto semielaborado o en producción
2062,HS-CHGQ_6813,NaN,Turno de mañana,NaN,NaN,45,Incidente,Incidente (no aplica),NaN,Área de embalado
2063,HS-CHGQ_6814,NaN,Turno de mañana,NaN,NaN,45,Incidente,Incidente (no aplica),NaN,Instalaciones exteriores y auxiliares
2064,HS-CHGQ_6138,NaN,Turno de mañana,NaN,NaN,45,Incidente,Incidente (no aplica),NaN,Puente Grúa
2065,HS-CHGQ_6139,NaN,Turno de tarde,NaN,NaN,45,Incidente,Incidente (no aplica),NaN,Instalaciones exteriores y auxiliares


Now we are going to upload the Plants and Tech file, which contains the relation between the fabric the tecnology and the country. We can join this table with the staff table by Planta - ID

In [60]:
planta = pd.read_csv('G_Plantas y Tech.csv', sep=';')
planta.drop(columns={'Activo'}, inplace=True)
planta.head()

,ID,Fabrica,Tecnología,Pais
0,1,Argentina,Blanking,Argentina
1,2,Argentina,Corte Longitudinal,Argentina
2,3,Argentina,Corte Transversal,Argentina
3,4,Arizona,Células Soldadura,EEUU
4,5,Arizona,Corte Láser,EEUU


In [61]:
# we merge both df so we have all information in staff df with a left join in case we dont have plants for some rows

staff = staff.merge(planta, left_on='Planta', right_on='ID', how='left')


In [62]:
staff.drop(columns={'ID'}, inplace=True)

staff.head()

,Codigo accidentado,Puesto habitual de trabajo,Turno,Edad,Género,Planta,Tipo accidente,Tipo trabajador,Gravedad potencial,Lugar o seccion accidente,Fabrica,Tecnología,Pais
0,HS-CHGQ_6253,NaN,Turno de mañana,NaN,NaN,45,Incidente,Incidente (no aplica),NaN,Almacén producto semielaborado o en producción,Asturias,Decapado,España
1,HS-CHGQ_6813,NaN,Turno de mañana,NaN,NaN,45,Incidente,Incidente (no aplica),NaN,Área de embalado,Asturias,Decapado,España
2,HS-CHGQ_6814,NaN,Turno de mañana,NaN,NaN,45,Incidente,Incidente (no aplica),NaN,Instalaciones exteriores y auxiliares,Asturias,Decapado,España
3,HS-CHGQ_6138,NaN,Turno de mañana,NaN,NaN,45,Incidente,Incidente (no aplica),NaN,Puente Grúa,Asturias,Decapado,España
4,HS-CHGQ_6139,NaN,Turno de tarde,NaN,NaN,45,Incidente,Incidente (no aplica),NaN,Instalaciones exteriores y auxiliares,Asturias,Decapado,España


In [63]:
len(staff) ## we have same amount as before

4180

Now we are going to upload some staff data to get the number of trainning hours per plant and have a reference of the quantity of employees we want to generate. 

This data is aggregated so it is not valid for our model.

In [64]:
rrhh_plant = pd.read_csv('Datos_plantilla_fabrica.csv', sep=';')

rrhh_plant['Año'].unique()

array([2017., 2016., 2018., 2019.,   nan, 2020., 2021.])

In [65]:
# we are getting only 2020 for our analysis to have a reference

rrhh_plant = rrhh_plant[rrhh_plant['Año']==2020]


In [66]:
rrhh_plant.head()

,ID,Planta,Año,Mes,Empleados Propios,Empleados Ajenos,Empleados ETTS,Empleados Subcontrata,Horas Presencia Efectiva Propios,Horas Presencia Efectiva Subcontratados,Horas Presencia Efectiva ETTs,Horas Formacion Seguridad Propios,Horas Formacion Seguridad ETTs,Horas Formacion General Propios,Empleados Propios - indefinido,Empleados Propios - duración determinada
2615,3819,Barcelona,2020.0,ENE,203.0,28.0,8.0,20.0,"29770,3",3138,"1307,5",15,23,211,189.0,14.0
2616,3820,Burgos,2020.0,ENE,239.0,68.0,28.0,40.0,32625,6487,"4549,3",445,"348,5",757,188.0,38.0
2617,3821,Tarragona,2020.0,ENE,49.0,17.0,5.0,12.0,"8352,5",2080,862,8,12,111,46.0,3.0
2618,3822,Valencia,2020.0,ENE,153.0,28.0,24.0,3.0,"22105,2",647,"3968,3",8,6,"35,5",139.0,2.0
2619,3823,Portugal,2020.0,ENE,99.0,12.0,10.0,2.0,"16625,75",384,"1866,5",80,"32,5",514,72.0,28.0


In [67]:
a = 545613.01
round(a)

545613

In [68]:
lista = ['Horas Presencia Efectiva Propios', 'Horas Presencia Efectiva Subcontratados', 'Horas Presencia Efectiva ETTs','Horas Formacion Seguridad Propios' ,'Horas Formacion Seguridad ETTs', 'Horas Formacion General Propios'  ]

for column in lista:  
  rrhh_plant[column] = rrhh_plant[column].apply(lambda x: x.replace(',', '.')).astype(float)
  rrhh_plant[column] = rrhh_plant[column].apply(lambda x: round(x))

In [69]:
rrhh_plant.dtypes

ID                                            int64
Planta                                       object
Año                                         float64
Mes                                          object
Empleados Propios                           float64
Empleados Ajenos                            float64
Empleados ETTS                              float64
Empleados Subcontrata                       float64
Horas Presencia Efectiva Propios              int64
Horas Presencia Efectiva Subcontratados       int64
Horas Presencia Efectiva ETTs                 int64
Horas Formacion Seguridad Propios             int64
Horas Formacion Seguridad ETTs                int64
Horas Formacion General Propios               int64
Empleados Propios - indefinido              float64
Empleados Propios - duración determinada    float64
dtype: object

In [70]:
trainning_hours = rrhh_plant[['Planta','Horas Presencia Efectiva Subcontratados', 'Horas Presencia Efectiva ETTs', 'Horas Formacion Seguridad Propios', 'Horas Formacion Seguridad ETTs']]

In [71]:
trainning_hours.rename(columns={'Planta':'Fabrica'}, inplace=True)
trainning_hours.head(1)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,Fabrica,Horas Presencia Efectiva Subcontratados,Horas Presencia Efectiva ETTs,Horas Formacion Seguridad Propios,Horas Formacion Seguridad ETTs
2615,Barcelona,3138,1308,15,23


In [72]:
trainning_hours = trainning_hours.groupby('Fabrica').agg('sum').reset_index() ## we get the nº of trainning hours by fabric
trainning_hours.head()

,Fabrica,Horas Presencia Efectiva Subcontratados,Horas Presencia Efectiva ETTs,Horas Formacion Seguridad Propios,Horas Formacion Seguridad ETTs
0,Addimen,0,0,0,0
1,Argentina,0,0,607,0
2,Arizona,0,0,0,0
3,Baja California,0,900,0,0
4,Barcelona,25685,18303,1151,152


In [73]:
staff = staff.merge(trainning_hours, on='Fabrica', how='left') # adding the training hours to staff df
staff.head()

,Codigo accidentado,Puesto habitual de trabajo,Turno,Edad,Género,Planta,Tipo accidente,Tipo trabajador,Gravedad potencial,Lugar o seccion accidente,Fabrica,Tecnología,Pais,Horas Presencia Efectiva Subcontratados,Horas Presencia Efectiva ETTs,Horas Formacion Seguridad Propios,Horas Formacion Seguridad ETTs
0,HS-CHGQ_6253,NaN,Turno de mañana,NaN,NaN,45,Incidente,Incidente (no aplica),NaN,Almacén producto semielaborado o en producción,Asturias,Decapado,España,NaN,NaN,NaN,NaN
1,HS-CHGQ_6813,NaN,Turno de mañana,NaN,NaN,45,Incidente,Incidente (no aplica),NaN,Área de embalado,Asturias,Decapado,España,NaN,NaN,NaN,NaN
2,HS-CHGQ_6814,NaN,Turno de mañana,NaN,NaN,45,Incidente,Incidente (no aplica),NaN,Instalaciones exteriores y auxiliares,Asturias,Decapado,España,NaN,NaN,NaN,NaN
3,HS-CHGQ_6138,NaN,Turno de mañana,NaN,NaN,45,Incidente,Incidente (no aplica),NaN,Puente Grúa,Asturias,Decapado,España,NaN,NaN,NaN,NaN
4,HS-CHGQ_6139,NaN,Turno de tarde,NaN,NaN,45,Incidente,Incidente (no aplica),NaN,Instalaciones exteriores y auxiliares,Asturias,Decapado,España,NaN,NaN,NaN,NaN


Now we are going to calculate the proportion in 2020 between people and accidents occur

In [74]:
employees = pd.read_csv('Datos_plantilla_fabrica.csv', sep=';')
employees = employees[['Año', 'Empleados Propios', 'Empleados Ajenos', 'Empleados ETTS',  'Empleados Subcontrata']]
employees = employees.fillna(0)
employees = employees.groupby(['Año']).agg('sum')
employees.head()

,Empleados Propios,Empleados Ajenos,Empleados ETTS,Empleados Subcontrata
Año,,,,
0.0,15754.0,3277.0,2490.0,786.0
2016.0,40609.0,9160.0,0.0,0.0
2017.0,43291.0,9794.0,0.0,0.0
2018.0,49865.0,10973.0,7964.0,2993.0
2019.0,50625.0,10527.0,7823.0,2697.0


In [75]:
accidentes = pd.read_csv('HS_Accidentabilidad.csv', sep=';')
accidentes = accidentes[['Año', 'Codigo accidentado']]
accidentes = accidentes.groupby('Año').agg('count').reset_index()
accidentes.rename(columns={'Codigo accidentado':'Accidentes'}, inplace=True)
accidentes.head()

,Año,Accidentes
0,2015.0,202
1,2016.0,1335
2,2017.0,1203
3,2018.0,1532
4,2019.0,1526


In [76]:
employees = employees.merge(accidentes, on='Año', how='inner')
employees.head()

,Año,Empleados Propios,Empleados Ajenos,Empleados ETTS,Empleados Subcontrata,Accidentes
0,2016.0,40609.0,9160.0,0.0,0.0,1335
1,2017.0,43291.0,9794.0,0.0,0.0,1203
2,2018.0,49865.0,10973.0,7964.0,2993.0,1532
3,2019.0,50625.0,10527.0,7823.0,2697.0,1526
4,2020.0,65644.0,7114.0,5773.0,1335.0,1122


In [77]:
employees.dtypes

Año                      float64
Empleados Propios        float64
Empleados Ajenos         float64
Empleados ETTS           float64
Empleados Subcontrata    float64
Accidentes                 int64
dtype: object

In [78]:
employees['Total_emp'] = employees['Empleados Propios'] + employees['Empleados Ajenos'] + employees['Empleados ETTS'] + employees['Empleados Subcontrata']

In [79]:
employees = employees[['Año', 'Accidentes', 'Total_emp']]

employees['perc_acc'] = employees['Accidentes'] / employees['Total_emp'] * 100
employees.head()

,Año,Accidentes,Total_emp,perc_acc
0,2016.0,1335,49769.0,2.682393
1,2017.0,1203,53085.0,2.266177
2,2018.0,1532,71795.0,2.133853
3,2019.0,1526,71672.0,2.129144
4,2020.0,1122,79866.0,1.404853


For our analysis we are going to suppose we have a year staff, with all people. 
We would have 79900 people with around 7000 people who have suffered an accident on the past years.

In [80]:
len(staff)

4180

In [81]:
#we add a new column to indetify those people who have suffered an accident in our dataset
staff['Accident'] = 'Yes'
staff.head()

,Codigo accidentado,Puesto habitual de trabajo,Turno,Edad,Género,Planta,Tipo accidente,Tipo trabajador,Gravedad potencial,Lugar o seccion accidente,Fabrica,Tecnología,Pais,Horas Presencia Efectiva Subcontratados,Horas Presencia Efectiva ETTs,Horas Formacion Seguridad Propios,Horas Formacion Seguridad ETTs,Accident
0,HS-CHGQ_6253,NaN,Turno de mañana,NaN,NaN,45,Incidente,Incidente (no aplica),NaN,Almacén producto semielaborado o en producción,Asturias,Decapado,España,NaN,NaN,NaN,NaN,Yes
1,HS-CHGQ_6813,NaN,Turno de mañana,NaN,NaN,45,Incidente,Incidente (no aplica),NaN,Área de embalado,Asturias,Decapado,España,NaN,NaN,NaN,NaN,Yes
2,HS-CHGQ_6814,NaN,Turno de mañana,NaN,NaN,45,Incidente,Incidente (no aplica),NaN,Instalaciones exteriores y auxiliares,Asturias,Decapado,España,NaN,NaN,NaN,NaN,Yes
3,HS-CHGQ_6138,NaN,Turno de mañana,NaN,NaN,45,Incidente,Incidente (no aplica),NaN,Puente Grúa,Asturias,Decapado,España,NaN,NaN,NaN,NaN,Yes
4,HS-CHGQ_6139,NaN,Turno de tarde,NaN,NaN,45,Incidente,Incidente (no aplica),NaN,Instalaciones exteriores y auxiliares,Asturias,Decapado,España,NaN,NaN,NaN,NaN,Yes


In [82]:
#plant data we dont have any null values
staff['Planta'].unique()

array([45, 30, 15,  4,  2, 20, 32, 25, 43,  5, 26, 33,  7, 53, 12,  1, 14,
        6, 41, 55, 34, 29, 13, 19, 28, 16, 36, 35, 54, 27, 57, 56, 18, 44,
       63, 46, 47, 84])

In [83]:
##work position

positions = staff['Puesto habitual de trabajo'].unique()
len(positions)

794

As we have 794 positions we are going to get only 50 to generate the random values:

In [84]:
positions = positions[1:50]
positions

array(['Anlagenführer Laser 5',
       'Gruista de caldera y limpieza de caldera',
       'Gruista de caldera y operadores de limpieza',
       'Operadores limpieza de caldera y gruista de caldera',
       'Auxiliares de Logistica\r\nOperador de Grua Telescopica',
       'Embalador 1 (en capacitación)', 'Empaquetador',
       'Maschinenbediener/-einrichter Presse 5', 'LONGI009',
       'Ninguno de los implicados desempeñaban funciones habituales en la línea donde se produjo el incidente.',
       'GRUISTA LONGI004-008+MANTENIMIENTO+MAQUINISTA LONGI008',
       'Auxiliar de producción',
       'Titular y ayudantes de Perfiladora de Dimeco Perfi012',
       'OPERARIOS DE CARGA', 'Maquinista perfiladora',
       'GRUISTA DE CALDERA Y OPERARIOS DE LIMPIEZA',
       'Embalaje del material en la línea de embalaje, cliente: Hydrapres S.A',
       'Gruista y limpieza de caldera', 'Operator / Stacker Operator',
       'Maquinista', 'Gruista ácido', 'Carretillero',
       'Director Comercial Pla

In [85]:
import random
import numpy as np 

staff['wk_position'] = np.random.choice(positions, size=len(staff))

In [86]:
staff['wk_position'].unique()

array(['Programador', 'Gruista y limpieza de caldera', 'Gruista ácido',
       'Montacarguista (en entrenamiento)',
       'Líneas de Corte Longitudinal',
       'Operadores limpieza de caldera y gruista de caldera', 'Maestro',
       'Maschinenbediener/-einrichter Presse 5', 'Maquinista perfiladora',
       'Anlagenführer Laser 5', 'OPERARIOS DE CARGA',
       'Gruista de caldera y limpieza de caldera', 'Cargador/descargador',
       'Flejador Línea Corte Prensa I', 'Líder de Logistica y Planeación',
       'Working on Press', 'Gruista Almacenes', 'Conductor Tren',
       'Ninguno de los implicados desempeñaban funciones habituales en la línea donde se produjo el incidente.',
       'Maquinista', 'Flejador-embalador',
       'Gruista de caldera y operadores de limpieza',
       'Gruista LONGI004 y LONGI008',
       'MAQUINISTA Y AYUDANTE TRANSVERSAL 4',
       'GRUISTA LONGI004-008+MANTENIMIENTO+MAQUINISTA LONGI008',
       'Flejador LONGI001', 'Camionero', 'Supervisor',
       'Direc

In [87]:
staff.drop(columns={'Puesto habitual de trabajo'}, inplace=True)

In [88]:
#gender

staff.rename(columns={'Género':'Gender'}, inplace=True)


In [89]:
staff['Gender'].unique()

array([nan, 'Hombre', 'Mujer'], dtype=object)

In [90]:
staff['Gender'] = staff['Gender'].fillna('Hombre')

In [91]:
len(staff[staff['Gender'].isnull()])

0

In [92]:
staff['Gender'].unique()

array(['Hombre', 'Mujer'], dtype=object)

Checking if we have null values in turno

In [93]:
staff.Turno.isnull().any()

False

Checking for nulls in age:

As we have null values we are goign to reassing the age range based on the proportion of the tables we hav analyzed

In [94]:
staff.Edad.isnull().any()

True

In [95]:
len(staff[staff.Edad.isnull()])

1855

In [96]:
rangos_edad = ['Menor 30', 'Entre 30-50', 'Mayor 50']
weights = {'Hombre':[0.2, 0.44, 0.36], 'Mujer':[0.2,0.55,0.25]}

In [97]:
np.random.seed(0)

def apply_randoms(x):
  key = x['Gender'].iat[0]
  return pd.Series(np.random.choice(rangos_edad, size=len(x), p=weights[key]))

In [98]:
staff['Age'] = staff.groupby('Gender').apply(apply_randoms)\
       .rename('Age').reset_index().drop('level_1', 1)['Age']

In [99]:
staff.drop(columns={'Edad'}, inplace=True)

Checking is there is any row with null at type of accident

In [100]:
staff['Tipo accidente'].isnull().any()

False

Worker type have null values at some rows

In [101]:
staff['Tipo trabajador'].isnull().any()

True

In [102]:
staff['Tipo trabajador'].unique()

array(['Incidente (no aplica)', 'ETT', 'Indefinido', 'Subcontratado',
       'Duración Determinada', 'Prácticas', nan,
       'Cedidos de otras plantas', 'No aplica', 'Indefinido con preaviso'],
      dtype=object)

In [103]:
staff[staff['Tipo trabajador'].isnull()] #As we only have two values null we are going to assign directly 'Indefinido'

,Codigo accidentado,Turno,Gender,Planta,Tipo accidente,Tipo trabajador,Gravedad potencial,Lugar o seccion accidente,Fabrica,Tecnología,Pais,Horas Presencia Efectiva Subcontratados,Horas Presencia Efectiva ETTs,Horas Formacion Seguridad Propios,Horas Formacion Seguridad ETTs,Accident,wk_position,Age
2831,HS-NVRR_8597,Turno de mañana,Hombre,13,Incidente,NaN,Leve,Carretilla,Burgos,Blanking,España,25249.0,42485.0,1419.0,829.0,Yes,MAQUINISTA Y AYUDANTE TRANSVERSAL 4,Menor 30
2832,HS-NVRR_8598,Turno de mañana,Hombre,13,Incidente,NaN,Leve,Carretilla,Burgos,Blanking,España,25249.0,42485.0,1419.0,829.0,Yes,Montacarguista (en entrenamiento),Mayor 50


In [104]:
staff['Tipo trabajador'] = staff['Tipo trabajador'].fillna('Indefinido')

In [105]:
staff['Tipo trabajador'].isnull().any()

False

Potential Severity columns has null values as well

In [106]:
staff[staff['Accident']=='Yes']['Gravedad potencial'].isnull().any()

True

In [107]:
len(staff[(staff['Gravedad potencial'].isnull()) & (staff['Accident']=='Yes')])

1009

In [108]:
## we are going to assign it randomly
staff['Gravedad potencial'].unique()
severity = ['Muy grave', 'Leve', 'Grave']

In [109]:
for i in staff['Gravedad potencial']:

  if str(i) == 'nan':
    staff['Severity'] = np.random.choice(severity, size=len(staff))
  else:
    staff['Severity'] = i


In [110]:
staff['Severity'].isnull().any()

False

In [111]:
staff['Severity'].unique()

array(['Leve', 'Muy grave', 'Grave'], dtype=object)

In [112]:
staff.drop(columns={'Gravedad potencial'}, inplace=True)

In [113]:
staff['Severity'].isnull().any()

False

Aciccent's place/section

In [114]:
staff['Lugar o seccion accidente'].isnull().any()

True

In [115]:
len(staff[staff['Lugar o seccion accidente'].isnull()])

63

In [116]:
staff['Lugar o seccion accidente'].unique()

array(['Almacén producto semielaborado o en producción',
       'Área de embalado', 'Instalaciones exteriores y auxiliares',
       'Puente Grúa', 'Corte Láser 3D', 'Galvanizado Inmersión',
       'Decapado', 'Blanking', 'Corte Transversal', 'Corte Longitudinal',
       'Almacén de Materia Prima', 'Perfiles abiertos', 'Cizalla',
       'Almacén de Producto Terminado', 'Tubos',
       'Oficinas administrativas', 'Prensa', 'Maquinaria Auxiliar',
       'Galvanizado continuo', 'Mantenimiento', 'Almacen de recambios',
       'Carretilla', 'Células Soldadura', 'Carpinteria', nan,
       'Corte Plasma', 'Corte Láser', 'Almacen productos quimicos',
       'Sierra', 'Punzonado', 'Soldadura Manual', 'Limpieza/Prepintado',
       'Perforado', 'Coil to Coil', 'Galvanizado ', 'Electrozincado',
       'Pintura', 'Tailor Welding', 'Galvanizado', 'General / Auxiliar',
       'Perfilado de barrera y postes', 'Skinpass', 'Slitting ',
       'General', 'Press Blanking', 'Ablation Line', 'Laminado',
    

In [117]:
place_acc = ['Células Soldadura', 'Tubos', 'Almacén de Producto Terminado',
       'Blanking', 'Corte Longitudinal', 'Corte Transversal',
       'Almacén de Materia Prima', 'Cizalla', 'Decapado', 'Puente Grúa',
       'Carpinteria', 'Galvanizado Inmersión',  'Maquinaria Auxiliar',
       'Corte Plasma', 'Perfiles abiertos', 'Corte Láser',
       'Almacen productos quimicos', 'Corte Láser 3D',
       'Instalaciones exteriores y auxiliares', 'Mantenimiento', 'Prensa',
       'Almacén producto semielaborado o en producción', 'Carretilla',
       'Sierra', 'Oficinas administrativas', 'Área de embalado',
       'Punzonado', 'Soldadura Manual', 'Limpieza/Prepintado',
       'Galvanizado continuo', 'Perforado', 'Coil to Coil',
       'Tailor Welding', 'Galvanizado ', 'Almacen de recambios',
       'Galvanizado', 'Electrozincado', 'Pintura', 'General / Auxiliar',
       'Perfilado de barrera y postes', 'Skinpass', 'Slitting ',
       'General', 'Press Blanking', 'Laminado', 'Ablation Line',
       'Slitting', 'D-Ring Line', 'Multi Strand']

In [118]:
for i in staff['Lugar o seccion accidente']:

  if str(i) == 'nan':
    staff['Accident_place'] = np.random.choice(place_acc, size=len(staff))
  else:
    staff['Accident_place'] = i

In [119]:
staff['Accident_place'].isnull().any()

False

In [120]:
staff.drop(columns={'Lugar o seccion accidente'}, inplace=True)

There arent any null values in Fabric column, technology and country fields are from fabric so the will be filled as well

In [121]:
staff['Fabrica'].isnull().any()

False

Now we are going to check the columns of efective presence hours

In [122]:
staff['Horas Presencia Efectiva Subcontratados'].isnull().any()

True

In [123]:
staff['Horas Presencia Efectiva Subcontratados'] = staff['Horas Presencia Efectiva Subcontratados'].fillna(0)

In [124]:
staff['Horas Presencia Efectiva ETTs'].isnull().any()

True

In [125]:
staff['Horas Presencia Efectiva ETTs'] = staff['Horas Presencia Efectiva ETTs'].fillna(0)

In [126]:
staff['Horas Formacion Seguridad Propios'].isnull().any()

True

In [127]:
staff['Horas Formacion Seguridad Propios'] = staff['Horas Formacion Seguridad Propios'].fillna(0)

In [128]:
staff['Horas Formacion Seguridad ETTs'].isnull().any()

True

In [129]:
staff['Horas Formacion Seguridad ETTs'] = staff['Horas Formacion Seguridad ETTs'].fillna(0)

In [130]:
staff.head()

,Codigo accidentado,Turno,Gender,Planta,Tipo accidente,Tipo trabajador,Fabrica,Tecnología,Pais,Horas Presencia Efectiva Subcontratados,Horas Presencia Efectiva ETTs,Horas Formacion Seguridad Propios,Horas Formacion Seguridad ETTs,Accident,wk_position,Age,Severity,Accident_place
0,HS-CHGQ_6253,Turno de mañana,Hombre,45,Incidente,Incidente (no aplica),Asturias,Decapado,España,0.0,0.0,0.0,0.0,Yes,Programador,Entre 30-50,Leve,Mantenimiento
1,HS-CHGQ_6813,Turno de mañana,Hombre,45,Incidente,Incidente (no aplica),Asturias,Decapado,España,0.0,0.0,0.0,0.0,Yes,Gruista y limpieza de caldera,Mayor 50,Muy grave,Mantenimiento
2,HS-CHGQ_6814,Turno de mañana,Hombre,45,Incidente,Incidente (no aplica),Asturias,Decapado,España,0.0,0.0,0.0,0.0,Yes,Gruista ácido,Entre 30-50,Muy grave,Mantenimiento
3,HS-CHGQ_6138,Turno de mañana,Hombre,45,Incidente,Incidente (no aplica),Asturias,Decapado,España,0.0,0.0,0.0,0.0,Yes,Montacarguista (en entrenamiento),Entre 30-50,Grave,Mantenimiento
4,HS-CHGQ_6139,Turno de tarde,Hombre,45,Incidente,Incidente (no aplica),Asturias,Decapado,España,0.0,0.0,0.0,0.0,Yes,Gruista y limpieza de caldera,Entre 30-50,Grave,Mantenimiento


Now we are going to generate the other part of our staff dataframe which contains those people who havent had an accident

In [131]:
len(staff)

4180

In [132]:
employees = rrhh_plant[['Año', 'Mes', 'Empleados Propios', 'Empleados Ajenos','Empleados ETTS','Empleados Subcontrata']]
employees  = employees[employees['Mes']=='ENE']

employees['Total'] = employees['Empleados Propios'] + employees['Empleados Ajenos'] + employees['Empleados ETTS'] + employees['Empleados Subcontrata']

employees = employees.groupby(['Año', 'Mes']).agg('sum')
employees['Total']

Año     Mes
2020.0  ENE    7070.0
Name: Total, dtype: float64

So the number of employees in 2020 7070 and we are going to take that as reference. 

As we are taking 3 years of accidents we are going to have the same proportion on the staff to be able to evaluate it. So our total number of employees in the staff table will be 21.210, 4180 accidents. 

We are going to generate now the 17030 employees who didnt have any accident

In [133]:
staff_without = pd.DataFrame(columns=staff.columns)

staff_without

,Codigo accidentado,Turno,Gender,Planta,Tipo accidente,Tipo trabajador,Fabrica,Tecnología,Pais,Horas Presencia Efectiva Subcontratados,Horas Presencia Efectiva ETTs,Horas Formacion Seguridad Propios,Horas Formacion Seguridad ETTs,Accident,wk_position,Age,Severity,Accident_place


In [134]:
def create_rndm(values, column):
  staff_without[column] = np.random.choice(values, size=17030)

In [135]:
turno  = ['Turno de mañana', 'Turno de tarde', 'Turno de noche']

create_rndm(turno, 'Turno')

To create the gender columns we are going to apply weights based on the data we had from staff

In [136]:
gender = ['Hombre', 'Mujer']
weights = [0.90,0.1]

np.random.seed(0)

staff_without['Gender'] = random.choices(gender, weights=weights, k=len(staff_without))

In [137]:
planta_list = staff.Planta.unique()
create_rndm(planta_list, 'Planta')

In [138]:
staff_without.head(1)

,Codigo accidentado,Turno,Gender,Planta,Tipo accidente,Tipo trabajador,Fabrica,Tecnología,Pais,Horas Presencia Efectiva Subcontratados,Horas Presencia Efectiva ETTs,Horas Formacion Seguridad Propios,Horas Formacion Seguridad ETTs,Accident,wk_position,Age,Severity,Accident_place
0,NaN,Turno de tarde,Hombre,45,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [139]:
staff_without.drop(columns={'Fabrica', 'Tecnología', 'Pais'}, inplace=True)

In [140]:
staff_without = staff_without.merge(planta, left_on='Planta', right_on='ID', how='left')
staff_without.head(2)

,Codigo accidentado,Turno,Gender,Planta,Tipo accidente,Tipo trabajador,Horas Presencia Efectiva Subcontratados,Horas Presencia Efectiva ETTs,Horas Formacion Seguridad Propios,Horas Formacion Seguridad ETTs,Accident,wk_position,Age,Severity,Accident_place,ID,Fabrica,Tecnología,Pais
0,NaN,Turno de tarde,Hombre,45,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,45,Asturias,Decapado,España
1,NaN,Turno de mañana,Hombre,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,Arizona,Células Soldadura,EEUU


In [141]:
len(staff_without)

17030

In [142]:
tipo_trabj = staff['Tipo trabajador'].unique()
create_rndm(tipo_trabj, 'Tipo trabajador')

In [143]:
staff_without.drop(columns=['Horas Presencia Efectiva Subcontratados','Horas Presencia Efectiva ETTs', 'Horas Formacion Seguridad Propios', 'Horas Formacion Seguridad ETTs' ], inplace=True)

In [144]:
staff_without = staff_without.merge(trainning_hours, on='Fabrica', how='left')

In [145]:
staff_without.drop(columns={'ID'}, inplace=True)

In [146]:
staff_without['Accident'] = 'No'

In [147]:
pos_wk = staff['wk_position'].unique()

create_rndm(pos_wk, 'wk_position')

In [148]:
rangos_edad = ['Menor 30', 'Entre 30-50', 'Mayor 50']
weights = {'Hombre':[0.2, 0.44, 0.36], 'Mujer':[0.2,0.55,0.25]}

np.random.seed(0)

staff_without['Age'] = staff_without.groupby('Gender').apply(apply_randoms)\
       .rename('Age').reset_index().drop('level_1', 1)['Age']

In [149]:
place = staff['Accident_place'].unique()

create_rndm(place, 'Accident_place')

In [150]:
staff_without.head(2) ##cod accidentado is the only column we are going to leave blank as these employees doestn have any accident

,Codigo accidentado,Turno,Gender,Planta,Tipo accidente,Tipo trabajador,Accident,wk_position,Age,Severity,Accident_place,Fabrica,Tecnología,Pais,Horas Presencia Efectiva Subcontratados,Horas Presencia Efectiva ETTs,Horas Formacion Seguridad Propios,Horas Formacion Seguridad ETTs
0,NaN,Turno de tarde,Hombre,45,NaN,Duración Determinada,No,LINEAS DE CORTE LONGITUDINAL,Entre 30-50,NaN,Mantenimiento,Asturias,Decapado,España,NaN,NaN,NaN,NaN
1,NaN,Turno de mañana,Hombre,4,NaN,Prácticas,No,plegador,Mayor 50,NaN,Mantenimiento,Arizona,Células Soldadura,EEUU,0.0,0.0,0.0,0.0


Now we are going to concat both dataframes on a resulting dataframe with length of 21.210

In [151]:
staff = pd.concat([staff, staff_without])
len(staff)

21210

In [152]:
staff.iloc[0]

Codigo accidentado                                  HS-CHGQ_6253
Turno                                            Turno de mañana
Gender                                                    Hombre
Planta                                                        45
Tipo accidente                                         Incidente
Tipo trabajador                            Incidente (no aplica)
Fabrica                                                 Asturias
Tecnología                                              Decapado
Pais                                                      España
Horas Presencia Efectiva Subcontratados                        0
Horas Presencia Efectiva ETTs                                  0
Horas Formacion Seguridad Propios                              0
Horas Formacion Seguridad ETTs                                 0
Accident                                                     Yes
wk_position                                          Programador
Age                      

In [153]:
from google.colab import files

staff.to_csv('Total_staff_by_employee.csv', sep=';')
files.download("Total_staff_by_employee.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>